# 피쳐를 벡터화 하는 과정

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pickle
import numpy as np

In [2]:
data = pd.read_csv('/opt/ml/input/final_project/Crawling/imdb_side_df_EmptyList_to_None.csv') # side information data
data = data.drop_duplicates('Contents',keep='first') # 중복 제거
data = data.drop(['produceers', 'composers'], axis=1) # 수집 하나도 안되서 쓸모 없음
data

,Contents,searched_content,genres,plot,directors,actors
0,Moana (2016),Moana,"['Animation', 'Adventure', 'Comedy', 'Family',...","[""In Ancient Polynesia, when a terrible curse ...","['Ron Clements', 'John Musker', 'Don Hall', 'C...","[""Auli'i Cravalho"", 'Dwayne Johnson', 'Rachel ..."
1,Inside Out (2015),Inside Out,"['Animation', 'Adventure', 'Comedy', 'Drama', ...",['After young Riley is uprooted from her Midwe...,"['Pete Docter', 'Ronnie Del Carmen']","['Amy Poehler', 'Phyllis Smith', 'Richard Kind..."
2,Zootopia (2016),Zootopia,"['Animation', 'Adventure', 'Comedy', 'Crime', ...","['In a city of anthropomorphic animals, a rook...","['Byron Howard', 'Rich Moore', 'Jared Bush']","['Ginnifer Goodwin', 'Jason Bateman', 'Idris E..."
3,Legally Blonde (2001),Legally Blonde,"['Comedy', 'Romance']",NaN,['Robert Luketic'],"['Reese Witherspoon', 'Luke Wilson', 'Selma Bl..."
4,Dead Poets Society (1989),Dead Poets Society,"['Comedy', 'Drama']",['Maverick teacher John Keating uses poetry to...,['Peter Weir'],"['Robin Williams', 'Robert Sean Leonard', 'Eth..."
...,...,...,...,...,...,...
4049,Cloak & Dagger (2018),Cloak & Dagger,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",['Two teenagers from very different background...,NaN,"['Olivia Holt', 'Aubrey Joseph', 'Gloria Reube..."
4050,Peacemaker (2022),Peacemaker,"['Action', 'Adventure', 'Comedy', 'Crime', 'Fa...",['Picking up where The Suicide Squad (2021) le...,NaN,"['John Cena', 'Danielle Brooks', 'Freddie Stro..."
4051,Guardians of the Galaxy (2015),Guardians of the Galaxy,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",['A group of intergalactic criminals must pull...,['James Gunn'],"['Chris Pratt', 'Zoe Saldana', 'Dave Bautista'..."
4052,Icon Comics,Iron Sky: The Coming Race,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",['A follow-up to the film Iron Sky (2012) in w...,['Timo Vuorensola'],"['Lara Rossi', 'Vladimir Burlakov', 'Kit Dale'..."


In [3]:
def remove_list_symbol(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['plot'][2:-2]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass
def remove_list_symbol_2(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['genres'][1:-1]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass

data['plot'] = data.apply(remove_list_symbol, axis=1)
data['genres'] = data.apply(remove_list_symbol_2, axis=1)
data

,Contents,searched_content,genres,plot,directors,actors
0,Moana (2016),Moana,"'Animation', 'Adventure', 'Comedy', 'Family', ...","In Ancient Polynesia, when a terrible curse in...","['Ron Clements', 'John Musker', 'Don Hall', 'C...","[""Auli'i Cravalho"", 'Dwayne Johnson', 'Rachel ..."
1,Inside Out (2015),Inside Out,"'Animation', 'Adventure', 'Comedy', 'Drama', '...",After young Riley is uprooted from her Midwest...,"['Pete Docter', 'Ronnie Del Carmen']","['Amy Poehler', 'Phyllis Smith', 'Richard Kind..."
2,Zootopia (2016),Zootopia,"'Animation', 'Adventure', 'Comedy', 'Crime', '...","In a city of anthropomorphic animals, a rookie...","['Byron Howard', 'Rich Moore', 'Jared Bush']","['Ginnifer Goodwin', 'Jason Bateman', 'Idris E..."
3,Legally Blonde (2001),Legally Blonde,"'Comedy', 'Romance'",None,['Robert Luketic'],"['Reese Witherspoon', 'Luke Wilson', 'Selma Bl..."
4,Dead Poets Society (1989),Dead Poets Society,"'Comedy', 'Drama'",Maverick teacher John Keating uses poetry to e...,['Peter Weir'],"['Robin Williams', 'Robert Sean Leonard', 'Eth..."
...,...,...,...,...,...,...
4049,Cloak & Dagger (2018),Cloak & Dagger,"'Action', 'Adventure', 'Drama', 'Sci-Fi'",Two teenagers from very different backgrounds ...,NaN,"['Olivia Holt', 'Aubrey Joseph', 'Gloria Reube..."
4050,Peacemaker (2022),Peacemaker,"'Action', 'Adventure', 'Comedy', 'Crime', 'Fan...",Picking up where The Suicide Squad (2021) left...,NaN,"['John Cena', 'Danielle Brooks', 'Freddie Stro..."
4051,Guardians of the Galaxy (2015),Guardians of the Galaxy,"'Action', 'Adventure', 'Comedy', 'Sci-Fi'",A group of intergalactic criminals must pull t...,['James Gunn'],"['Chris Pratt', 'Zoe Saldana', 'Dave Bautista'..."
4052,Icon Comics,Iron Sky: The Coming Race,"'Action', 'Adventure', 'Comedy', 'Sci-Fi'",A follow-up to the film Iron Sky (2012) in whi...,['Timo Vuorensola'],"['Lara Rossi', 'Vladimir Burlakov', 'Kit Dale'..."


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_em_vec(sample):
    input_ids = torch.tensor(tokenizer.encode(sample)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    em_vec = last_hidden_states[0][0]
    return outputs

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df_dict = {'Contents':[], 'genre_vec':[]}

In [6]:
pbar = tqdm(list(zip(data['Contents'], data['genres'])))
for c,g in pbar:
    df_dict['Contents'].append(c)
    try:
        if len(g)>512:
            g_emv = get_em_vec(g[:513])
            g_emv = g_emv[0][0][0].detach().numpy()
            df_dict['genre_vec'].append(g_emv[0][0][0])
        else:
            g_emv = get_em_vec(g)
            df_dict['genre_vec'].append(g_emv[0][0][0])
    except TypeError:
        df_dict['genre_vec'].append(torch.zeros(768))


# t.detach().numpy()

100%|██████████| 4035/4035 [02:28<00:00, 27.20it/s]


In [7]:
with open('bert_em_plot_vec_dict_2.pkl','wb') as f:
    pickle.dump(df_dict, f)